In [1]:
import pandas as pd
import yfinance as yf
from datetime import timedelta
import pandas_ta as ta


In [2]:
tickers = ['TSLA', 'RIVN', 'NVDA','AAPL']
years = 5

In [3]:
def data_download_daily(tickers, years):
    today = pd.Timestamp.today()
    days_in_years = 365*years
    start_date = (today - timedelta(days=days_in_years)).strftime('%Y-%m-%d')
    end_date = today
    days_in_years = 365*years
    prices_df = pd.DataFrame()
    for ticker in tickers:
        data = yf.download(ticker, start = start_date, end= end_date, interval='1D')
        prices_df[ticker] = data['Adj Close']
    return prices_df


In [4]:
prices_df = data_download_daily(tickers=tickers, years=years)
prices_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,TSLA,RIVN,NVDA,AAPL
Date,,,,
2019-03-11 00:00:00-04:00,19.394667,NaN,40.011406,43.056702
2019-03-12 00:00:00-04:00,18.890667,NaN,40.354057,43.540466
2019-03-13 00:00:00-04:00,19.264000,NaN,41.868706,43.732994
2019-03-14 00:00:00-04:00,19.330667,NaN,41.108906,44.219162
2019-03-15 00:00:00-04:00,18.362000,NaN,42.164188,44.794376
...,...,...,...,...
2024-03-01 00:00:00-05:00,202.639999,11.35,822.751404,179.660004
2024-03-04 00:00:00-05:00,188.139999,10.91,852.330017,175.100006
2024-03-05 00:00:00-05:00,180.740005,10.93,859.640015,170.119995


In [5]:
def calculate_my_strategy(input_df):
    strat = input_df.copy()
    for ticker in input_df.columns:
        strat[ticker + '_upper_50'] =   strat[ticker].rolling(window = 45).mean() + abs(strat[ticker].rolling(window = 90).mean())*0.5
        strat[ticker + '_lower_50'] =   strat[ticker].rolling(window = 45).mean() - abs(strat[ticker].rolling(window = 90).mean())*0.5
        strat[ticker + '_rolling_50'] = strat[ticker].rolling(window = 50).mean()
    return strat

In [6]:
my_strat = calculate_my_strategy(prices_df)


In [7]:
#input dataframe has to have tickers and column names whoch contain daily pices 
def calculate_donchian(prices_df):
    donchian_df = pd.DataFrame(index=prices_df.index)  # DataFrame to store Donchian channels
    for ticker in prices_df.columns:
        donchian_df[[f"{ticker}_dcl",f"{ticker}_dcm", f"{ticker}_dcu"]] = prices_df.ta.donchian(high = prices_df[ticker], low = prices_df[ticker], lower_length = 40, upper_length = 40)
        donchian_df[f'{ticker}_upper_dcu'] = donchian_df[f'{ticker}_dcu'] - (donchian_df[f'{ticker}_dcu'] - donchian_df[f'{ticker}_dcm'])*0.35
        donchian_df[f'{ticker}_lower_dcl'] = donchian_df[f'{ticker}_dcl'] + (donchian_df[f'{ticker}_dcm'] - donchian_df[f'{ticker}_dcl'])*0.35

    return donchian_df

In [8]:
donchian_df = calculate_donchian(prices_df)
donchian_df.tail(5)

,TSLA_dcl,TSLA_dcm,TSLA_dcu,TSLA_upper_dcu,TSLA_lower_dcl,RIVN_dcl,RIVN_dcm,RIVN_dcu,RIVN_upper_dcu,RIVN_lower_dcl,NVDA_dcl,NVDA_dcm,NVDA_dcu,NVDA_upper_dcu,NVDA_lower_dcl,AAPL_dcl,AAPL_dcm,AAPL_dcu,AAPL_upper_dcu,AAPL_lower_dcl
Date,,,,,,,,,,,,,,,,,,,,
2024-03-01 00:00:00-05:00,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,14.825,19.58,17.91575,11.73425,479.957489,651.354446,822.751404,762.762469,539.946424,179.660004,187.295631,194.931259,192.258789,182.332473
2024-03-04 00:00:00-05:00,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,14.825,19.58,17.91575,11.73425,490.946960,671.638489,852.330017,789.087982,554.188995,175.100006,185.015633,194.931259,191.460790,178.570475
2024-03-05 00:00:00-05:00,180.740005,210.595001,240.449997,230.000748,191.189254,10.07,14.825,19.58,17.91575,11.73425,522.505493,691.072754,859.640015,800.641473,581.504034,170.119995,182.525627,194.931259,190.589288,174.461966
2024-03-06 00:00:00-05:00,176.539993,205.750000,234.960007,224.736504,186.763496,10.07,14.660,19.25,17.64350,11.67650,531.375122,709.187561,887.000000,824.765646,593.609476,169.119995,182.025627,194.931259,190.414288,173.636966
2024-03-07 00:00:00-05:00,176.539993,205.239998,233.940002,223.895001,186.584995,10.07,14.570,19.07,17.49500,11.64500,543.474487,735.082245,926.690002,859.627287,610.537202,169.000000,181.965630,194.931259,190.393289,173.537970


In [9]:
prices_df

,TSLA,RIVN,NVDA,AAPL
Date,,,,
2019-03-11 00:00:00-04:00,19.394667,NaN,40.011406,43.056702
2019-03-12 00:00:00-04:00,18.890667,NaN,40.354057,43.540466
2019-03-13 00:00:00-04:00,19.264000,NaN,41.868706,43.732994
2019-03-14 00:00:00-04:00,19.330667,NaN,41.108906,44.219162
2019-03-15 00:00:00-04:00,18.362000,NaN,42.164188,44.794376
...,...,...,...,...
2024-03-01 00:00:00-05:00,202.639999,11.35,822.751404,179.660004
2024-03-04 00:00:00-05:00,188.139999,10.91,852.330017,175.100006
2024-03-05 00:00:00-05:00,180.740005,10.93,859.640015,170.119995


In [10]:
def backtest(reference, ticker, trades):
    max_drawdown=0
    longest_hold=0
    shortest_hold=0
    average_drawdown=0
    max_gain=0
    total_gain= 0

My strategy

In [11]:
merge = pd.merge(prices_df, donchian_df, how='left', left_index=True, right_index=True)
merge

,TSLA,RIVN,NVDA,AAPL,TSLA_dcl,TSLA_dcm,TSLA_dcu,TSLA_upper_dcu,TSLA_lower_dcl,RIVN_dcl,...,NVDA_dcl,NVDA_dcm,NVDA_dcu,NVDA_upper_dcu,NVDA_lower_dcl,AAPL_dcl,AAPL_dcm,AAPL_dcu,AAPL_upper_dcu,AAPL_lower_dcl
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-11 00:00:00-04:00,19.394667,NaN,40.011406,43.056702,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-12 00:00:00-04:00,18.890667,NaN,40.354057,43.540466,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-13 00:00:00-04:00,19.264000,NaN,41.868706,43.732994,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-14 00:00:00-04:00,19.330667,NaN,41.108906,44.219162,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-15 00:00:00-04:00,18.362000,NaN,42.164188,44.794376,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01 00:00:00-05:00,202.639999,11.35,822.751404,179.660004,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,...,479.957489,651.354446,822.751404,762.762469,539.946424,179.660004,187.295631,194.931259,192.258789,182.332473
2024-03-04 00:00:00-05:00,188.139999,10.91,852.330017,175.100006,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,...,490.946960,671.638489,852.330017,789.087982,554.188995,175.100006,185.015633,194.931259,191.460790,178.570475
2024-03-05 00:00:00-05:00,180.740005,10.93,859.640015,170.119995,180.740005,210.595001,240.449997,230.000748,191.189254,10.07,...,522.505493,691.072754,859.640015,800.641473,581.504034,170.119995,182.525627,194.931259,190.589288,174.461966


1. if lower donchian indicator is crossed from bottom hold until it crosses donchian upper indicator
2. if upper donchian indicator is crossed from the top, hold until its crossed from below the lower donchian indicator 

long position column where it says true if open position is opened

short position column where it says true if short postion is opened 

obviously both of them are opened as long as they are not closed 

for my case when one is closed second is opened

longest position is when you have the most consecutive true values for either of those columns

biggest drawdown is when you have the biggest difference form the last highest position in the position, if it dips after peaking during long position or peaks after dipin during a short position

average drawdown is sum of all drawdowns devided by the sum of drawdowns (non 0 values in drawdown column)

max gain and average gain is the opposite of drawdowns

all of these need separate columns

In [12]:
backtest_df = merge.copy()

In [13]:
backtest_df

,TSLA,RIVN,NVDA,AAPL,TSLA_dcl,TSLA_dcm,TSLA_dcu,TSLA_upper_dcu,TSLA_lower_dcl,RIVN_dcl,...,NVDA_dcl,NVDA_dcm,NVDA_dcu,NVDA_upper_dcu,NVDA_lower_dcl,AAPL_dcl,AAPL_dcm,AAPL_dcu,AAPL_upper_dcu,AAPL_lower_dcl
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-11 00:00:00-04:00,19.394667,NaN,40.011406,43.056702,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-12 00:00:00-04:00,18.890667,NaN,40.354057,43.540466,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-13 00:00:00-04:00,19.264000,NaN,41.868706,43.732994,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-14 00:00:00-04:00,19.330667,NaN,41.108906,44.219162,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-15 00:00:00-04:00,18.362000,NaN,42.164188,44.794376,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01 00:00:00-05:00,202.639999,11.35,822.751404,179.660004,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,...,479.957489,651.354446,822.751404,762.762469,539.946424,179.660004,187.295631,194.931259,192.258789,182.332473
2024-03-04 00:00:00-05:00,188.139999,10.91,852.330017,175.100006,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,...,490.946960,671.638489,852.330017,789.087982,554.188995,175.100006,185.015633,194.931259,191.460790,178.570475
2024-03-05 00:00:00-05:00,180.740005,10.93,859.640015,170.119995,180.740005,210.595001,240.449997,230.000748,191.189254,10.07,...,522.505493,691.072754,859.640015,800.641473,581.504034,170.119995,182.525627,194.931259,190.589288,174.461966


jak zrobic tak zeby zapisac kiedy otwarcie pozycji, zapamietywalo ostatnia pozycje, jak jest otwarta to nic nie dodaje

In [31]:
def donchian_check(tickers, input_df):
    column_1 = '_lower_dcl'
    column_2 = '_upper_dcu'
    backtest_df = input_df.copy()
    pos_status_long = False
    pos_status_short = False
    for ticker in tickers:
        for date in backtest_df.index:
            if date == backtest_df.index[0]:
                continue
            

            i = backtest_df.index.get_loc(date)
            
            if backtest_df[ticker].iloc[i-1] < backtest_df[ticker + column_1].iloc[i-1] and backtest_df[ticker].iloc[i] >= backtest_df[ticker + column_1].iloc[i]:
                backtest_df.at[date, ticker +'_long_pos'] = 'open_long_position'
                backtest_df.at[date, ticker + '_short_pos'] = 'close_short_position'
                backtest_df.at[date, ticker +'_starting_price'] = backtest_df.at[date, ticker]
            if backtest_df[ticker].iloc[i-1] > backtest_df[ticker + column_2].iloc[i-1] and backtest_df[ticker].iloc[i] < backtest_df[ticker + column_2].iloc[i]:
                backtest_df.at[date, ticker +'_long_pos'] = 'close_long_position'
                backtest_df.at[date, ticker + '_short_pos'] = 'open_short_position'
                backtest_df.at[date, ticker +'_starting_price'] = backtest_df.at[date, ticker]
    return backtest_df

In [39]:
def stats_test(tickers, input_df):
    backtest_df = input_df.copy()
    for ticker in tickers:
        backtest_df[ticker + '_long_pos'] = backtest_df[ticker + '_long_pos'].fillna(method='ffill')
        backtest_df[ticker + '_short_pos'] = backtest_df[ticker + '_short_pos'].fillna(method='ffill')
        backtest_df[ticker + '_starting_price'] = backtest_df[ticker + '_starting_price'].fillna(method='ffill')
        for date in backtest_df.index:
            if date == backtest_df.index[0]:
                continue
            

            i = backtest_df.index.get_loc(date)

            if backtest_df[ticker + '_long_pos'].iloc[i]== 'open_long_position':
                backtest_df[ticker + 'pos_pct_change'] = (backtest_df[ticker] - backtest_df[ticker + '_starting_price'])/backtest_df[ticker + '_starting_price']
                if backtest_df[ticker + 'pos_pct_change'].iloc[i] > -0.15:
                    backtest_df[ticker + '_long_pos'] == 'close_long_position'
            if backtest_df[ticker + '_short_pos'].iloc[i] == 'open_short_position':
                backtest_df[ticker + 'pos_pct_change'] = (backtest_df[ticker + '_starting_price'] - backtest_df[ticker])/backtest_df[ticker + '_starting_price']
                if backtest_df[ticker + 'pos_pct_change'].iloc[i] > -0.15:
                    backtest_df[ticker + '_long_pos'] == 'close_short_position'

    return backtest_df


In [40]:
donchian_df = donchian_check(tickers, backtest_df)
donchian_df

,TSLA,RIVN,NVDA,AAPL,TSLA_dcl,TSLA_dcm,TSLA_dcu,TSLA_upper_dcu,TSLA_lower_dcl,RIVN_dcl,...,TSLA_starting_price,RIVN_long_pos,RIVN_short_pos,RIVN_starting_price,NVDA_long_pos,NVDA_short_pos,NVDA_starting_price,AAPL_long_pos,AAPL_short_pos,AAPL_starting_price
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-11 00:00:00-04:00,19.394667,NaN,40.011406,43.056702,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-12 00:00:00-04:00,18.890667,NaN,40.354057,43.540466,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-13 00:00:00-04:00,19.264000,NaN,41.868706,43.732994,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-14 00:00:00-04:00,19.330667,NaN,41.108906,44.219162,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-15 00:00:00-04:00,18.362000,NaN,42.164188,44.794376,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01 00:00:00-05:00,202.639999,11.35,822.751404,179.660004,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-04 00:00:00-05:00,188.139999,10.91,852.330017,175.100006,181.059998,210.754997,240.449997,230.056747,191.453247,10.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-05 00:00:00-05:00,180.740005,10.93,859.640015,170.119995,180.740005,210.595001,240.449997,230.000748,191.189254,10.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
stats_df = stats_test(tickers, donchian_df)
stats_df.tail(50)

,TSLA,RIVN,NVDA,AAPL,TSLA_dcl,TSLA_dcm,TSLA_dcu,TSLA_upper_dcu,TSLA_lower_dcl,RIVN_dcl,...,NVDA_long_pos,NVDA_short_pos,NVDA_starting_price,AAPL_long_pos,AAPL_short_pos,AAPL_starting_price,TSLApos_pct_change,RIVNpos_pct_change,NVDApos_pct_change,AAPLpos_pct_change
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-26 00:00:00-05:00,256.609985,23.590000,492.766907,192.803986,197.360001,227.290001,257.220001,246.744501,207.835501,15.33,...,close_long_position,open_short_position,481.087402,close_long_position,open_short_position,192.803986,0.195481,0.238971,-0.024277,0.000000
2023-12-27 00:00:00-05:00,261.440002,23.730000,494.146820,192.903839,200.839996,231.139999,261.440002,250.835001,211.444997,15.33,...,close_long_position,open_short_position,481.087402,close_long_position,open_short_position,192.803986,0.217983,0.246323,-0.027146,0.000518
2023-12-28 00:00:00-05:00,253.179993,23.540001,495.196777,193.333298,205.660004,233.550003,261.440002,251.678503,215.421503,15.33,...,close_long_position,open_short_position,481.087402,close_long_position,open_short_position,192.803986,0.179502,0.236345,-0.029328,0.002745
2023-12-29 00:00:00-05:00,248.479996,23.459999,495.196777,192.284637,209.979996,235.709999,261.440002,252.434501,218.985497,15.33,...,close_long_position,open_short_position,481.087402,close_long_position,open_short_position,192.803986,0.000000,0.232143,-0.029328,-0.002694
2024-01-02 00:00:00-05:00,248.419998,21.100000,481.657410,185.403412,209.979996,235.709999,261.440002,252.434501,218.985497,15.33,...,close_long_position,open_short_position,481.657410,close_long_position,open_short_position,192.803986,-0.000241,0.000000,0.000000,-0.038384
2024-01-03 00:00:00-05:00,238.449997,20.320000,475.667694,184.015198,209.979996,235.709999,261.440002,252.434501,218.985497,15.33,...,close_long_position,open_short_position,481.657410,close_long_position,open_short_position,192.803986,-0.040365,-0.036967,0.012436,-0.045584
2024-01-04 00:00:00-05:00,237.929993,19.500000,479.957489,181.678177,209.979996,235.709999,261.440002,252.434501,218.985497,15.33,...,close_long_position,open_short_position,481.657410,close_long_position,open_short_position,192.803986,-0.042458,-0.075829,0.003529,-0.057705
2024-01-05 00:00:00-05:00,237.490005,19.080000,490.946960,180.949097,209.979996,235.709999,261.440002,252.434501,218.985497,15.33,...,close_long_position,open_short_position,481.657410,close_long_position,open_short_position,192.803986,-0.044229,-0.095735,-0.019287,-0.061487
2024-01-08 00:00:00-05:00,240.449997,19.580000,522.505493,185.323517,209.979996,235.709999,261.440002,252.434501,218.985497,15.33,...,close_long_position,open_short_position,481.657410,open_long_position,close_short_position,185.323517,-0.032316,-0.072038,-0.084807,0.000000


In [43]:
aapl = stats_df.filter(regex = 'AAPL')
aapl.tail(50)

,AAPL,AAPL_dcl,AAPL_dcm,AAPL_dcu,AAPL_upper_dcu,AAPL_lower_dcl,AAPL_long_pos,AAPL_short_pos,AAPL_starting_price,AAPLpos_pct_change
Date,,,,,,,,,,
2023-12-26 00:00:00-05:00,192.803986,169.849197,183.853363,197.857529,192.956071,174.750655,close_long_position,open_short_position,192.803986,0.000000
2023-12-27 00:00:00-05:00,192.903839,170.327972,184.092751,197.857529,193.039856,175.145645,close_long_position,open_short_position,192.803986,0.000518
2023-12-28 00:00:00-05:00,193.333298,173.519684,185.688606,197.857529,193.598406,177.778807,close_long_position,open_short_position,192.803986,0.002745
2023-12-29 00:00:00-05:00,192.284637,176.192749,187.025139,197.857529,194.066192,179.984085,close_long_position,open_short_position,192.803986,-0.002694
2024-01-02 00:00:00-05:00,185.403412,176.192749,187.025139,197.857529,194.066192,179.984085,close_long_position,open_short_position,192.803986,-0.038384
2024-01-03 00:00:00-05:00,184.015198,178.766068,188.311798,197.857529,194.516523,182.107073,close_long_position,open_short_position,192.803986,-0.045584
2024-01-04 00:00:00-05:00,181.678177,181.349365,189.603447,197.857529,194.968600,184.238294,close_long_position,open_short_position,192.803986,-0.057705
2024-01-05 00:00:00-05:00,180.949097,180.949097,189.403313,197.857529,194.898553,183.908072,close_long_position,open_short_position,192.803986,-0.061487
2024-01-08 00:00:00-05:00,185.323517,180.949097,189.403313,197.857529,194.898553,183.908072,open_long_position,close_short_position,185.323517,0.000000


In [ ]:
# def create_gains_df(tickers, input_df):
#     gains_df = pd.DataFrame()
#     for ticker in tickers:
#         temporary = input_df[(input_df[f'{ticker}_long_pos'].str.contains('position', na=False)) | 
#                     (input_df[f'{ticker}_short_pos'].str.contains('position', na=False))][[ticker, f'{ticker}_long_pos', f'{ticker}_short_pos']]
#         gains_df = pd.concat([gains_df, temporary])
#     return gains_df

In [ ]:
# gains_df = create_gains_df(tickers, donchian_df)
# gains_df

In [ ]:
temp_df = donchian_df.filter(regex='TSLA')
temp_df

1. can define a function for one and just reapeat for every ticker
2. figure out if i want to return multiple dataframes or find a way to return results for multiple tickers


In [ ]:
def bbacktest(tickers, backtest_df):
        results_list = []
        #iterate through tickers
        # Assuming your DataFrame is called 'temporary_df'
        for ticker in tickers:
                temporary_df = pd.DataFrame()
                temporary_df = backtest_df.filter(regex = ticker)
                long_position = False
                open_date_long = []
                close_date_long = []
                open_price_long = []
                close_price_long = []
                results_long = pd.DataFrame()
                results_short = pd.DataFrame()
                #create lists of positions to be able to track and summerize them
                #make sure columns that will be checked are in correct formats
                pos_columns = [col for col in temporary_df.columns if 'pos' in col]
                temporary_df[pos_columns] = temporary_df[pos_columns].astype(str)

                #first calculate long positions
                for date, row in temporary_df.iterrows():
                        if long_position == False and 'open' in row[f'{ticker}_long_pos']:
                                open_date_long.append(date)
                                open_price_long.append(row[ticker])
                                long_position = True
                        if long_position == True and 'close' in row[f'{ticker}_long_pos']:
                                close_date_long.append(date)
                                close_price_long.append(row[ticker])
                                long_position = False
                if len(open_date_long) > len(close_date_long):
                        close_date_long.append(pd.Timestamp.today())
                if len(open_price_long) > len(close_price_long):
                        close_price_long.append(backtest_df[ticker][-1]) 
                results_long = pd.DataFrame({'open_date': open_date_long, 'close_date': close_date_long, 'open_price':open_price_long, 'close_price':close_price_long})
                results_long['gain'] = (results_long['close_price'] - results_long['open_price']) / results_long['open_price']
                results_long['total'] = None
                results_long['position'] = 'Long'

                #repeat for short positions
                short_position = False
                open_date_short  = []
                close_date_short = []
                open_price_short = []
                close_price_short = []
                for date, row in temporary_df.iterrows():
                        if short_position == False and 'open' in row[f'{ticker}_short_pos']:
                                open_date_short.append(date)
                                open_price_short.append(row[ticker])
                                short_position = True
                        if short_position == True and 'close' in row[f'{ticker}_short_pos']:
                                close_date_short.append(date)
                                close_price_short.append(row[ticker])
                                short_position = False
                if len(open_date_short) > len(close_date_short):
                        close_date_short.append(pd.Timestamp.today())
                if len(open_price_short) > len(close_price_short):
                        close_price_short.append(backtest_df[ticker][-1]) 
                results_short = pd.DataFrame({'open_date': open_date_short, 'close_date': close_date_short, 'open_price':open_price_short, 'close_price':close_price_short})
                results_short['gain'] = (results_short['close_price'] - results_short['open_price']) / results_short['open_price']
                results_short['total'] = None
                results_short['position'] = 'Short'
                total_results = pd.concat([results_long, results_short], axis=0)
                total_results = total_results.sort_values(by = 'open_date', ascending=True)
                total_results.reset_index(inplace = True,drop=True)
                for i in range(len(total_results['gain'])):
                        if i == 0:
                                total_results['total'][i] = 100 + total_results['gain'][i] * 100
                        else:
                                total_results['total'][i] = total_results['total'][i-1] + total_results['total'][i-1] * total_results['gain'][i]
                results_list.append(total_results)
        return results_list
#    return total_results
        


naprawic kierunek, ceny i 'tagi' transakcji

!!!!!!!

In [ ]:
results = bbacktest(tickers, donchian_df)


In [ ]:
results[0]

dodac stop loss na 10 lub 15%

In [ ]:
rivn = donchian_df.filter(regex='RIVN')


In [ ]:
rivn.columns

In [ ]:
import matplotlib.pyplot as plt

# Plot the graph with customized colors and transparency
rivn_plot = rivn[['RIVN', 'RIVN_dcl', 'RIVN_dcm', 'RIVN_dcu','RIVN_upper_dcu', 'RIVN_lower_dcl']]
#rivnl_plot = aapl_plot[aapl_plot.index >= '2022-01-01']
rivn_plot['RIVN'].plot(color='darkblue', figsize=(10, 6), label='RIVN')
rivn_plot['RIVN_dcl'].plot(color = 'lightblue', label = 'donchian_aapl_dcl', alpha = 0.5)
rivn_plot['RIVN_dcu'].plot(color = 'lightblue', label = 'donchian_aapl_dcu', alpha = 0.5)
rivn_plot['RIVN_upper_dcu'].plot(color = 'green', label = 'donchain_upper_indicator', alpha = 0.5)
rivn_plot['RIVN_lower_dcl'].plot(color = 'red', label = 'donchain_lower_indicator',alpha = 0.5)


# Add legend
plt.legend()

# Add title and labels
plt.title('RIVN Data with Donchian Channels')
plt.xlabel('Date')
plt.ylabel('Price')

# Show the plot
plt.show()


przyjrzeć się kilku caseom z bliska i uleprzyć warunki